In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import (Settings, VectorStoreIndex, SimpleDirectoryReader, PromptTemplate)
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.node_parser import SentenceSplitter
import chromadb

Import LLM

In [2]:
llm = Ollama(model="llama3")

Import embedding model

In [3]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = llm
Settings.embed_model = embed_model

c:\Users\Diana\Documents\Code\RAG\ollama\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Import documents and read them

In [4]:
documents = SimpleDirectoryReader(input_files=["./Diana_Morales_Resume.pdf"]).load_data()

In [5]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("ollama")

In [6]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, 
                                        storage_context=storage_context, 
                                        embed_model=embed_model,
                                        transformations=[SentenceSplitter(chunk_size=256, chunk_overlap=10)])

Prompting

In [7]:
# custome prompt template
template = (
    "Imagine you are a data scientit's assistant and you answer recruiter's questions about the data scientist's experience."
    "Here is some context from the data scientist's resume related to the query::\n"
    "-----------------------------------------\n"
    "{context_str}\n"
    "-----------------------------------------\n"
    "Considering the above information, please respond to the following inquiry:\n\n"
    "Question: {query_str}\n\n"
    "Answer succinctly and ensure your response is understandable to someone without data science background."
    "The data scientist's name is Diana."
)
qa_template = PromptTemplate(template)

# build query engine with custom template
# text_qa_template specifies custom template
# similarity_top_k configure the retriever to return the top 3 most similar documents,
# the default value of similarity_top_k is 2
query_engine = index.as_query_engine(text_qa_template=qa_template, similarity_top_k=3)

In [10]:
response = query_engine.query("Do you have experience with Python?")

In [11]:
response.response

'Yes, Diana has experience working with Python. According to her resume, Python is listed as one of her skills under the "Skills" section. This suggests that she has some proficiency in using Python for various tasks, possibly including data analysis, machine learning, and more.'